# TEST

Nueva agregación

Intenamos subir la accuracy del modelo federado

In [ ]:
from skmultilearn.dataset import load_dataset
from RepartoDatos import RepartoDatos
from keras.callbacks import EarlyStopping
import keras
from keras.layers import Dense
import matplotlib.pyplot as plt

In [ ]:
from MLFedAvg import MLFedAvg
from funciones_aux import *
from Metricas import Metrica


## CARGA DE DATOS

In [ ]:

from skmultilearn.dataset import load_from_arff

filename = 'SLASHDOT-F.arff'
q = 22

X, y = load_from_arff(
    filename,
    label_count=q,
    label_location='start',
    load_sparse=True # depende del conjunto de datos
)

X_array = X.toarray() 
y_array = y.toarray()
print(X_array.shape)
X=pd.DataFrame(X_array)
print(y_array.shape)
y=pd.DataFrame(y_array)



In [ ]:

# # Cargar el dataset emotions completo (undivided), del repositorio que tiene la libreria:
# #   http://scikit.ml/datasets.html#scikit-multilearn-repository
# X_sparse, y_sparse, feature_names, label_names = load_dataset('scene', 'undivided')
# X_array = X_sparse.toarray() # Esto nos da un array de numpy 
# print(X_array.shape)
# X=pd.DataFrame(X_array, columns=[str(feature) for feature in feature_names])


# label_names_B = [s for s,v in label_names]
# label_names_B
# y_array = y_sparse.toarray()
# print(y_array.shape)
# y=pd.DataFrame(y_array, columns=label_names_B)


## ENTRENAMIENTO FEDERADO

### Parámetros federado

In [ ]:
N_clientes=4
epochs_por_ronda=1 #epochs que ejecuta cada cliente en cada ronda de federado
semilla_aleatoria=1
reparto_etiquetas=[10,6,4,2]#etiquetas para cada cliente
N_rondas_federado=1000
agregacion=0

In [ ]:
if len(reparto_etiquetas) != N_clientes:
    raise ValueError("Error: El número de clientes no se corresponde con el reparto de etiquetas")


if sum(reparto_etiquetas) != y.shape[1]:
    raise ValueError("El número de etiquetas del dataset no se corresponde con el reparto de etiquetas")


### Partición de datos y etiquetas

Dividimos para obtener un conjunto de test global para probar nuestros modelos al finalizar el entrenamiento de aprendizaje federado. Preparamos los datos eliminando ceros y repartiendo dataset y etiquetas.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=semilla_aleatoria)
X_train,y_train=eliminar_ceros([X_train],[y_train])
X_test,y_test=eliminar_ceros([X_test],[y_test]) #conjunto de test global, para evaluación final
reparto=RepartoDatos(X_train[0],y_train[0])
XC_raw,yC_raw=reparto.dividir_dataset(N_clientes,random_state=semilla_aleatoria) #XC es una lista con los X de todos los clientes, igual con y
yC_raw_etiquetas=reparto.dividir_etiquetas(yC_raw,reparto_etiquetas,aleatorio=True,random_state=semilla_aleatoria)

XC_raw son los daset de cada cliente en bruto. yC_raw tiene las instancias de cada cliente (las que corresponden a XC) pero todos los clientes tienen todas las etiquetas. Ahora las dividimos, en orden o aleatorio.

Ahora eliminamos las filas las cuales sus etiquetas asociadas son todas 0. Estas serían mis listas con los datos finales.

In [ ]:
XC,yC=eliminar_ceros(XC_raw,yC_raw_etiquetas)

Añadimos ruido

In [ ]:
ruidoC=lista_aleatoria(N_clientes,0,0.5,semilla_aleatoria)
XC_ruido=ruido_lista(XC,ruidoC,semilla_aleatoria)

### Creación del modelo CORE

Creo un modelo con tantas neuronas de entrada como atributos tienen mis datos, y con las neuronas que le pase en la lista y creamos el objeto métricas.

In [ ]:
neuronas_core=[XC[0].shape[1],100,20] 

modelo=MLFedAvg(neuronas_core,S=epochs_por_ronda)
modelo_core=modelo.model_core()

metricas=Metrica()

Cada elemento es el numero de neuronas de cada capa. El modelo core tendrá tantas capas como elementos la lista (numero de capas=longitud de la lista)

Dividimos los datos de cada cliente en entrenamiento y test. Estos conjuntos son propios de cada cliente.

In [ ]:
XC_train_ruido,yC_train,XC_test,yC_test=train_test_split_list(XC_ruido,yC,test_size=0.2,random_state=semilla_aleatoria)


In [ ]:
instancias_cliente_train=cuenta_instancias(XC_train_ruido)

In [ ]:
instancias_cliente_train

### Entrenamiento federado

In [ ]:
def entrenamiento_federado(N_rondas,modelo_core,umbral_parada=10):

    ultimos_accuracy_entrenamiento = []
    primeros_accuracy_entrenamiento = []
    ultimos_accuracy_validacion = []
    primeros_accuracy_validacion = []
    ultimos_loss_validacion = []
    primeros_loss_validacion = []
    early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=False)
    N_ronda_stop=N_rondas
    medias_stop_loss=[]
    medias_stop_accuracy=[]

    
    for i in range(N_rondas):
        print("\n")
        print("-----------------------------------------------------------------------------------------")
        print("-----------------------------RONDA {}/{} DE APRENDIZAJE FEDERADO-------------------------".format(i+1,N_rondas))
        print("-----------------------------------------------------------------------------------------")

        
        modelos_clientes=modelo.create_client_model(modelo_core,yC,show_summ=False)
            
        if i>0:
            modelos_clientes=modelo.aplicar_pesos_ultima_capa(modelos_clientes,pesos_ultima_capa)

        modelos_compilados=modelo.compilar_modelos(modelos_clientes,lr=0.0008,metrics=['accuracy'],loss="binary_crossentropy")
        modelos_entrenados,accuracy_train, accuracy_val, loss_val, early_stopped=modelo.entrenar_modelos(modelos_compilados, 
                                                                                                  XC_train_ruido,yC_train,callbacks=[early_stopping_monitor],val_split=0.1,draw=False)        
        pesos_ultima_capa=modelo.extraer_pesos_ultima_capa(modelos_entrenados)

        for nombre_modelo, accuracy_train in accuracy_train.items():
            primeros_accuracy_entrenamiento.append((nombre_modelo, accuracy_train[0]))
            ultimos_accuracy_entrenamiento.append((nombre_modelo, accuracy_train[-1]))
            
        for nombre_modelo, accuracy_val in accuracy_val.items():
            primeros_accuracy_validacion.append((nombre_modelo, accuracy_val[0]))
            ultimos_accuracy_validacion.append((nombre_modelo, accuracy_val[-1]))

        for nombre_modelo, loss_val in loss_val.items():
            primeros_loss_validacion.append((nombre_modelo, loss_val[0]))
            ultimos_loss_validacion.append((nombre_modelo, loss_val[-1]))
        
        pesos_por_modelo=modelo.extraer_pesos_intermedios(modelos_entrenados)
        pesos_agregados=modelo.agregar_pesos(pesos_por_modelo,instancias_cliente_train,reparto_etiquetas,agg=agregacion)
        modelo_core_actualizado=modelo.aplicar_pesos_a_modelo(modelo_core,pesos_agregados)
        modelo_core=modelo_core_actualizado

        ultimos_accuracy_validacion_dic = {}
        ultimos_loss_validacion_dic = {}

        for cliente, numero in ultimos_accuracy_validacion:
            if cliente in ultimos_accuracy_validacion_dic:
                ultimos_accuracy_validacion_dic[cliente].append(numero)
            else:
                ultimos_accuracy_validacion_dic[cliente] = [numero]

        for cliente, numero in ultimos_loss_validacion:
            if cliente in ultimos_loss_validacion_dic:
                ultimos_loss_validacion_dic[cliente].append(numero)
            else:
                ultimos_loss_validacion_dic[cliente] = [numero]
        
        deberia_detenerse=False
        
        '''
        for cliente, valores in ultimos_accuracy_validacion_dic.items():
            print(valores)
            if len(valores) >= umbral_parada:
                if valores[-1] > valores[-2]:
                    deberia_detenerse=False

       
          
        if deberia_detenerse==True:
            for cliente, valores in ultimos_loss_validacion_dic.items():
                if len(valores) >= umbral_parada:
                    if valores[-1] < valores[-2]: 
                        deberia_detenerse=False
             '''       
        todos_los_valores_loss=[]
        todos_los_valores_accuracy=[]

        for valores_lista in ultimos_loss_validacion_dic.values():
            todos_los_valores_loss.extend(valores_lista)

        for valores_lista in ultimos_accuracy_validacion_dic.values():
            todos_los_valores_accuracy.extend(valores_lista)
        

        media_loss=np.mean(todos_los_valores_loss)
        media_accuracy=np.mean(todos_los_valores_accuracy)

        medias_stop_loss.append(media_loss)
        medias_stop_accuracy.append(media_accuracy)

        if i >= umbral_parada and medias_stop_loss[-1]>medias_stop_loss[-2] and medias_stop_loss[-2]>medias_stop_loss[-3] and medias_stop_accuracy[-1]<medias_stop_accuracy[-2] and medias_stop_accuracy[-2]<medias_stop_accuracy[-3]:
            deberia_detenerse=True

        print(medias_stop_loss)
        print(medias_stop_accuracy)
        if deberia_detenerse and i>=umbral_parada:
            N_ronda_stop = i
            break
           


    return modelos_entrenados,ultimos_accuracy_entrenamiento,ultimos_accuracy_validacion,primeros_accuracy_entrenamiento,primeros_accuracy_validacion, ultimos_accuracy_validacion_dic,N_ronda_stop

In [ ]:
modelos_entrenados,ultimos_accuracy_entrenamiento,ultimos_accuracy_validacion,primeros_accuracy_entrenamiento,primeros_accuracy_validacion, ultimos_accuracy_validacion_dic,N_ronda_stop=entrenamiento_federado(N_rondas_federado,modelo_core,umbral_parada=10)

## RENDIMIENTO

### Rendimiento sobre el conjunto de test

Probamos ahora el rendimiento de estos modelos entrenados con el conjunto de test global:

Primero creamos los y_test global de cada cliente, con sus etiquetas.

In [ ]:
yC_test_global=obtener_etiquetas_test_global(yC_train,y_test)

In [ ]:
X_test_global_sin_Ceros,yC_test_global_sin_ceros=eliminar_ceros(X_test*N_clientes,yC_test_global)

In [ ]:
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL SIN CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados,X_test_global_sin_Ceros,yC_test_global_sin_ceros)
print()
print()
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL CON CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados,X_test,yC_test_global)
print()
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST PROPIO")
evaluar_modelos(modelos_entrenados,XC_test,yC_test)


In [ ]:
#dibujar_graficas(ultimos_accuracy_entrenamiento,ultimos_accuracy_validacion,primeros_accuracy_entrenamiento, primeros_accuracy_validacion)

### Comparación rendimientos

#### Modelos propios de cada cliente sin federado

Cada modelo con sus datos sin rondas federadas, con epochs=Numero de rondas*épocas por donda

In [ ]:
modelo_sin_federado=MLFedAvg(neuronas_core,S=N_ronda_stop*epochs_por_ronda)
modelo_core_sin_federado=modelo_sin_federado.model_core()


In [ ]:
early_stopping_monitor_TD = EarlyStopping(monitor='val_loss', patience=N_ronda_stop, verbose=1,restore_best_weights=False)


modelos_clientes_sin_federado=modelo_sin_federado.create_client_model(modelo_core_sin_federado,yC_train,show_summ=False)
modelos_compilados_sin_federado=modelo_sin_federado.compilar_modelos(modelos_clientes_sin_federado,lr=0.01,metrics=['accuracy',metricas.JS()],loss="binary_crossentropy")
modelos_entrenados_sin_federado,ultimo_accuracy_train, ultimo_accuracy_val,ltimo_val_loss,a=modelo_sin_federado.entrenar_modelos(modelos_compilados_sin_federado, 
                                                                                                  XC_train_ruido,yC_train,callbacks=[early_stopping_monitor_TD],val_split=0.1,draw=False)        
                

In [ ]:
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL SIN CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados_sin_federado,X_test_global_sin_Ceros,yC_test_global_sin_ceros)

print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL CON CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados_sin_federado,X_test,yC_test_global)

print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST PROPIO")
evaluar_modelos(modelos_entrenados_sin_federado,XC_test,yC_test)


#### Único modelo con todos los datos

In [ ]:
modelo_unico=MLFedAvg(neuronas_core,S=N_rondas_federado*epochs_por_ronda)
modelo_core_unico=modelo_unico.model_core()
modelo_core_unico.add(Dense(y.shape[1],activation="sigmoid"))
modelo_core_unico.summary()

In [ ]:
early_stopping_modelo_unico = EarlyStopping(monitor='val_loss', patience=N_ronda_stop, verbose=1,restore_best_weights=True)

modelo_core_unico.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=["binary_crossentropy"], metrics=["accuracy"])

historia=modelo_core_unico.fit(X_train[0],y_train[0],epochs=N_rondas_federado*epochs_por_ronda,batch_size=30,callbacks=[early_stopping_modelo_unico],validation_split=0.2)        
plt.plot(historia.history["accuracy"], label='train')
plt.plot(historia.history[f'val_{"accuracy"}'], label='validation')
plt.xlabel('epoch')
plt.ylabel("accuracy")
plt.legend()
plt.show()

#### RESUMEN

In [ ]:
print("=========================================================================")
print("=========================================================================")

print("RESULTADOS PARA {} CLIENTES, EPOCHS={}  ,  ETIQUETAS: {} ,  RUIDO:{} ,  SEMILLA={}, AGREGACION={}".format(N_clientes,epochs_por_ronda,reparto_etiquetas,np.round(ruidoC,3),semilla_aleatoria,agregacion))
print()
print("MODELOS ENTRENADOS CON SUS DATOS PROPIOS MEDIANTE APRENDIZAJE FEDERADO")
print("==========================================================================")
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL SIN CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados,X_test_global_sin_Ceros,yC_test_global_sin_ceros)
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL CON CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados,X_test,yC_test_global)
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST PROPIO")
evaluar_modelos(modelos_entrenados,XC_test,yC_test)
print()
print()
print("MODELOS ENTRENADOS CON SUS DATOS PROPIOS SIN APRENDIZAJE FEDERADO")
print("==========================================================================")
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL SIN CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados_sin_federado,X_test_global_sin_Ceros,yC_test_global_sin_ceros)
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST GLOBAL CON CEROS EN LAS ETIQUETAS")
evaluar_modelos(modelos_entrenados_sin_federado,X_test,yC_test_global)
print("EVALUACIÓN MODELOS CON CONJUNTO DE TEST PROPIO")
evaluar_modelos(modelos_entrenados_sin_federado,XC_test,yC_test)
print()
print()
print("MODELO CENTRAL ÚNICO CON TODOS LOS DATOS")
print("=========================================================================")
evalua_modelo_unico(modelo_core_unico,X_test,y_test)


In [ ]:
import os

# Aquí iría el código que ejecuta tus celdas de Jupyter

# Al final de la ejecución, mostrar una notificación
os.system("osascript -e 'display notification \"Celdas de Jupyter terminadas\"'")
